In [ ]:
import requests
from bs4 import *
import openpyxl
from datetime import datetime
from tqdm import tqdm

In [ ]:


class Medic():

    def __init__(self,headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
):
        self.url = f"https://www.1mg.com"
        self.headers = headers

    def get_dis_link(self,lab):
        url = f"{self.url}/all-diseases?label={lab}"
        req = requests.get(url=url,headers=self.headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        data = []
        divtag = soup.find('div',attrs={'class':'style__product-grid___3noQW'})

        for div in divtag:
            anchor = div.find('a',attrs={'class':'style__product-name___HASYw'})
            med_link = anchor['href']
            data.append(med_link)

        global start
        start = datetime.now()
        print(data)
        self.get_all_rows(data)


    def get_all_rows(self,data):
        for link in tqdm(data):
            try:
                title , desc , sym , caus , prev , home_remedy = self.get_disease(med_link=link)
                self.set_excel(title , desc , sym , caus , prev , home_remedy)
            except Exception as e:
                print(e)
                print("NOT FOUND  : "+link)

    def get_disease(self,med_link):
      url = f"https://www.1mg.com{med_link}"
      req = requests.get(url=url,headers=self.headers)
      soup = BeautifulSoup(req.content, 'html.parser')
      title = ''
      desc = ''
      title_pane = soup.find('div',attrs={'class':'col-6 marginTop-16'})
      r = []

      for i in title_pane:
        title = i.find('h1')
        if title!=None:
            title = title.get_text(strip=True)
            break

      desc = title_pane.find('div', attrs={'id':'overview'})
      if desc!= None:
          desc = desc.get_text(strip=True).replace("Overview","")


      sym = title_pane.find('div', attrs={'id':'symptoms'})
      sym_list = []
      if sym!= None:
          sym_list_inter = sym.find_all('h3')
          for sym in sym_list_inter:
            sym = sym.get_text(strip=True)
            sym_list.append(sym)

      caus = title_pane.find('div', attrs={'id':'causes'})
      caus_list = []
      if caus!= None:
          caus_list_inter = caus.find_all('li')
          for caus in caus_list_inter:
            caus = caus.get_text(strip=True)
            caus_list.append(caus)


      prev = title_pane.find('div', attrs={'id':'prevention'})
      prev_list = []
      if prev!= None:
          prev_list_inter = prev.find_all('h3')
          for prev in prev_list_inter:
            prev = prev.get_text(strip=True)
            prev_list.append(prev)


      home_remedy = title_pane.find('div', attrs={'id':'alternatives_therapies'})
      home_remedy_list = []
      if home_remedy!= None:
          home_remedy_list_inter = home_remedy.find_all('strong')
          for home_remedy in home_remedy_list_inter:
            home_remedy = home_remedy.get_text(strip=True)
            home_remedy_list.append(home_remedy)

      return str(title) , str(desc) , str(sym_list) , str(caus_list) , str(prev_list) , str(home_remedy_list)

    def set_excel(self, title , desc , sym , caus , prev , home_remedy):
      sheet.append([ title , desc , sym , caus , prev , home_remedy])




In [ ]:
label_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
label_list = ['x','y','z']
excel = openpyxl.Workbook()
sheet = excel.active
title = f'Medic_disease'
sheet.title = title
sheet.append(["title" , "desc" , "sym" , "caus" , "prev" , "home_remedy"])
start = 0
obj = Medic()
for label in label_list:
  obj.get_dis_link(label)

excel.save(f'/content/drive/MyDrive/MEDIC/medic_disease_2.xlsx')

end = datetime.now()-start
print(end)

In [ ]:
excel.save(f'/content/drive/MyDrive/MEDIC/medic_disease.xlsx')


In [ ]:
# prompt: open excel file , make json of each row with keys of column name and value is that rows cell.

import openpyxl
import json

def excel_to_json(excel_file_path):
  workbook = openpyxl.load_workbook(excel_file_path)
  sheet = workbook.active
  header_row = [cell.value for cell in sheet[1]]
  data_rows = [row for row in sheet.iter_rows(min_row=2, values_only=True)]
  json_data = []
  for row in data_rows:
    row_dict = dict(zip(header_row, row))
    json_data.append(row_dict)
  return json_data

# Replace 'your_excel_file.xlsx' with the actual path to your Excel file
excel_file_path = '/content/drive/MyDrive/MEDIC/medic_disease_2.xlsx'
json_data = excel_to_json(excel_file_path)

# Replace 'your_output_file.json' with the desired path and filename
output_file_path = '/content/drive/MyDrive/MEDIC/medic_disease_2.json'

with open(output_file_path, 'w') as f:
  json.dump(json_data, f, indent=2)


# To save the JSON data to a file:
# with open('your_output_file.json', 'w') as f:
#   json.dump(json_data, f, indent=2)
